# Panoptic segmentation using OpenVINO™

Understand the _state-of-the-art_ computer vision technology which enables driverless vehicles to perceive their environments in real time and helps it make reasonable decisions while driving!! 

We will be consulting this [repo](https://github.com/hustvl/YOLOP) for the code, models and other data assets. At the same time we will learn how to use the OpenVINO [API](https://docs.openvino.ai/nightly/notebooks/003-hello-segmentation-with-output.html) for converting a model in a different format to the standard OpenVINO™ IR model representation.

### Imports 

In [19]:
import sys
from pathlib import Path
from typing import Optional,Union
from openvino.runtime import Core, serialize

sys.path.append("../utils")
import notebook_utils as utils

### Loading an ONXX model

[ONNX](https://onnx.ai/) is an open format built to represent machine learning models. ONNX defines a common set of operators - the building blocks of machine learning and deep learning models - and a common file format to enable AI developers to use models with a variety of frameworks, tools, runtimes, and compilers. OpenVINO supports reading models in ONNX format directly,that means they can be used with OpenVINO Runtime without any prior conversion.

Reading and loading an ONNX model, which is a single .onnx file, works the same way as with an OpenVINO IR model. The model argument points to the filename of an ONNX model. 

**The YoLoP models we would be dealing with today are indeed .onxx model files optimised for different image sizes**. We shall directly point to this directory for acquiring the model(s): [https://github.com/hustvl/YOLOP/blob/d37e600cf71ecac20b08865ddfe923d76fd02d55/weights/](https://github.com/hustvl/YOLOP/blob/d37e600cf71ecac20b08865ddfe923d76fd02d55/weights/)

In [13]:
# Create a model directory adjacent to the notebook and suppress errors if the directory already exists
Path('model').mkdir(exist_ok=True)

# Download the model file corresponding to the one trained for images of size 320 x 320 from the link 
# into the directory created above. Suppress errors if the file already exists
utils.download_file("https://github.com/hustvl/YOLOP/blob/main/weights/yolop-320-320.onnx", directory='model', silent=True) 

# Uncomment the following lines if you want to download the remaining models as well

# utils.download_file("https://github.com/hustvl/YOLOP/blob/main/weights/yolop-640-640.onnx", directory='model', silent=True) 
# utils.download_file("https://github.com/hustvl/YOLOP/blob/main/weights/yolop-1280-1280.onnx", directory='model', silent=True) 

model\yolop-320-320.onnx: 0.00B [00:00, ?B/s]

WindowsPath('G:/PRONOY/openvino_notebooks/notebooks/408-YoLoP-semantic-segmentation-webcam/model/yolop-320-320.onnx')

### OpenVINO™ magic 🪄

This is what we have come here for. Now we shall convert the downloaded model into the OpenVINO [IR](https://docs.openvino.ai/nightly/notebooks/002-openvino-api-with-output.html#onnx-model) representation by first compiling it on our device and then serializing the same into the ```.xml``` format. We will take one more step and delete the ```.onxx``` file which was used to do the same. The entirety of the process shall be defined in a custom function as below.

In [21]:
ie = Core()

def convert_to_ir_and_remove(model_path: str, compile_device: Union[str, int] = "CPU") -> None:
    onnx_model_path = Path(model_path)
    
    if not Path(onnx_model_path.stem + '.xml').is_file():
        
        model_onnx_yolop = ie.read_model(model=onnx_model_path)

        # Compilation has been done on the default CPU device. This may be quick for loading models, however, GPUs perform the best when it comes to actual inference.
        compiled_model_onnx_yolop = ie.compile_model(model=model_onnx_yolop, device_name=compile_device)
        
        serialize(compiled_model_onnx_yolop, xml_path=Path(onnx_model_path.stem + '.xml'))
    
    
    # Remove the .onxx file if it exists
    # suppress errors if the file is not found
    onnx_model_path.unlink(missing_ok=True)


In [25]:
onnx_model_path = "model/yolop-320-320.onnx"
model_onnx = ie.read_model(model=onnx_model_path)
# compiled_model_onnx = ie.compile_model(model=model_onnx, device_name=0)

RuntimeError: [ NETWORK_NOT_READ ] Unable to read the model: model/yolop-320-320.onnx Please check that model format: onnx is supported and the model is correct. Available frontends: ir onnx paddle tf 

In [22]:
convert_to_ir_and_remove('model/yolop-320-320.onxx')

RuntimeError: Model file model\yolop-320-320.onxx cannot be opened!

### Data loading

This is a critical step. It is known that as per the [guidelines](https://github.com/pronoym99/openvino_notebooks/blob/main/CONTRIBUTING.md#file-structure), ```data``` may be loaded into a directory of the same name adjacent to the [README](README.md) file and must not be large in size. Keeping in mind that the creation of this directory is an optional step, we will directly use the link to the [data](https://github.com/hustvl/YOLOP/tree/main/pictures) to get our job done and avoid creation of unnecessary directories when the images and videos are openly accessible across the internet.